<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment1_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading dataset

In [1]:
import tarfile
my_tar = tarfile.open('/content/lingspam_public.tar.gz')
my_tar.extractall('/content/') # specify which folder to extract to
my_tar.close()

In [12]:
import os
train_path = '/content/lingspam_public/lemm_stop/part1'  # for training      #spams: spmsg*.txt
test_path = '/content/lingspam_public/lemm_stop/part1'   # for testing

def to_dict (path):
  print('number of files:', len(os.listdir(path)))
  data_dict = dict()
  data_dict['spam'] = []
  data_dict['not_spam'] = []
  for file in os.listdir(path):
    
    doc = open (path + '/'+ file, 'r')
    if file == 'spmsg*.*':
      data_dict['spam'].append(doc)
    else:
      data_dict['not_spam'].append(doc)
  return data_dict

training = to_dict (train_path)
testing  = to_dict (test_path)

number of files: 289
number of files: 289


In [14]:
training.items()

dict_items([('spam', []), ('not_spam', [<_io.TextIOWrapper name='/content/lingspam_public/lemm_stop/part1/5-1227msg1.txt' mode='r' encoding='UTF-8'>, <_io.TextIOWrapper name='/content/lingspam_public/lemm_stop/part1/spmsga138.txt' mode='r' encoding='UTF-8'>, <_io.TextIOWrapper name='/content/lingspam_public/lemm_stop/part1/5-1245msg2.txt' mode='r' encoding='UTF-8'>, <_io.TextIOWrapper name='/content/lingspam_public/lemm_stop/part1/5-1239msg1.txt' mode='r' encoding='UTF-8'>, <_io.TextIOWrapper name='/content/lingspam_public/lemm_stop/part1/3-380msg2.txt' mode='r' encoding='UTF-8'>, <_io.TextIOWrapper name='/content/lingspam_public/lemm_stop/part1/5-1236msg1.txt' mode='r' encoding='UTF-8'>, <_io.TextIOWrapper name='/content/lingspam_public/lemm_stop/part1/3-437msg5.txt' mode='r' encoding='UTF-8'>, <_io.TextIOWrapper name='/content/lingspam_public/lemm_stop/part1/3-559msg3.txt' mode='r' encoding='UTF-8'>, <_io.TextIOWrapper name='/content/lingspam_public/lemm_stop/part1/5-1261msg1.txt' mo